## Ch.2 Working with Text Data

- The full notebook can be accessed via: https://github.com/rasbt/LLMs-from-scratch/blob/main/ch02/01_main-chapter-code/ch02.ipynb

In [7]:
import re

### 1. Load Data

In [4]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Total num of character: ", len(raw_text))
print(raw_text[:99])

Total num of character:  20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


### 2. Vocabulary

- A vocabulary is the set of all possible tokens that the model can recognize and process.
- Let's construct a vocabulary by creating a dictionary with unique tokens as the keys and Token IDs as the values

In [19]:
# Tokenize
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [22]:
# create a vocabulary
all_words = sorted(set(preprocessed)) # sorted list of unique words
vocab_size = len(all_words)

print("Vocab size: ", vocab_size)

vocab = {token: integer for integer,token in enumerate(all_words)}

Vocab size:  1130


In [23]:
for i,item in enumerate(vocab.items()):
    print(item)
    if i >= 5:
        break # print only the first 5 entries

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)


### 3. SimpleTokenizer

In [24]:
class SimpleTokenizer:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int 
            else "<|unk|>" for item in preprocessed # <|unk|> is for unknown token
        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

In [25]:
tokenizer = SimpleTokenizer(vocab)

In [26]:
text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


## 4. BytePair Encoding

- GPT-2 used BytePair encoding (BPE) as its tokenizer
- It allows the model to break down words that aren't in its predefined vocabulary into smaller subword units or even individual characters, enabling it to handle out-of-vocabulary words